# Memory Management in HIPFort

## Introduction
When working on GPU accelerated platforms there are different types of memory that you need to be aware of. This includes

* Host memory
* Global (Device) memory
* Texture memory
* Constant memory
* Shared memory
* Private/local memory

If you've worked with Fortran previously, host memory is already a familiar concept. Functionally, it is the data that you have declared in Fortran and possibly allocated space for with an `allocate` statement. When programming  GPUs, **Host memory** is the slowest memory to access on the GPU since host memory must be transfered to **Global (Device) memory** across a PCIe switch or other hardware fabric. HIP and HIPFort provide two strategies for managing host and device data. You can either explicitly declare and manage host and device pointers in your application or you can use **managed memory**. Explicit memory management provides you with detailed control over memory allocation and data movement between host and device, but it is verbose and increases the number of lines of code for an application significantly. **Managed memory** on the other hand allows you to keep track of only one pointer that can be accessed on either the host or device. To control data movement between host and device, you can provide "advice" on data locality in different regions of your code and request data pre-fetching to or from the device when needed.

**Texture memory** is fast read-only memory useful for working with images and is located on the compute device in fast cache. **Shared** and **constant memory** are usually located in small and fast caches on the compute device. Private memory is usually located in the registers, which is the fastest and smallest memory space available on the compute device. Local memory (not to be confused with the OpenCL term for shared memory) is private to the kernel, but uses the same memory hardware as global memory. A programmer has some degree of control over where memory is stored during the operation of a HIPFort program. The diagram below shows what memory is available for access by both host and kernel threads at runtime.

<figure style="margin-left:auto; margin-right:auto; width:80%;">
    <img style="vertical-align:middle" src="../images/memory_spaces.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Access to memory from kernel and host threads.</figcaption>
</figure>

## Managing Host and Device Memory

When working with HIPFort, your application needs to manage both host and device memory. In this section, we will cover
* Memory allocation and deallocation for host and device memory
* Explicit memory movement between host and device
* Pinned and pageable host memory
* Managed memory allocation, deallocation, and data movement between host and device

### Explicit Memory management

#### Memory allocation and deallocation

No matter which route you choose to manage your host and device data, you can use either the Fortran `pointer` or `type(c_ptr)` for referencing host or device data. It is important to mention that host data **cannot** be declared as a Fortran `allocatable`. Typically, host data is allocated by using the `allocate` intrinsic in Fortran and is deallocated with `deallocate`. More details on working with data in Fortran are covered in the [Fortran Refresher](../Fortran_Refresher/Fortran.ipynb).

```fortran

  implicit none
  
  integer, parameter :: N=1000
  real, pointer, dimension(:) :: f_host
  
  allocate(f_host(1:N))
  
  ...
  
  deallocate(f_host)
```

Device data is allocated using the `hipMalloc` function and can be deallocated (or free'd) using `hipFree`. The `hipMalloc` function takes in a pointer that will be used to reference device memory and the amount of memory we want to allocate. For the device pointer, we can use either a Fortran `pointer` or `type(c_ptr)` for the device data. When using the Fortran `pointer`, we have a few options for  specifying the amount of memory we want to allocate. We can explictly provide the number of elements of the array that we want to allocate, 

```fortran
  use hipfort
  use hipfort_check
  
  implicit none
  
  integer, parameter :: N=1000
  real, pointer, dimension(:) :: f_host
  real, pointer, dimension(:) :: f_device
  
  allocate(f_host(1:N))
  call hipcheck(hipMalloc(f_device, SIZE(f_host))
  
  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```

Here, the `SIZE` intrinsic is used to determine how many elements there are in `f_host`. This value is passed as the dimension of `f_device`. For multi-dimensional arrays, we can use the `SHAPE` intrinsic instead.

```fortran
  use hipfort
  use hipfort_check
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  real, pointer, dimension(:,:) :: f_device
  
  allocate(f_host(1:N,1:M))
  call hipcheck(hipMalloc(f_device, SHAPE(f_host))
  
  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```


Alternatively, the `mold` argument can be used to obtain array shape and size information from the host-side counterpart for our device data. Under the hood, the HIPFort API will obtain the dimensions of the host data and will use this to calculate the size and shape of the device pointer. The benefit of using this optional argument is that it will work for single and multi-dimensional arrays.

```fortran
  use hipfort
  use hipfort_check
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  real, pointer, dimension(:,:) :: f_device
  
  allocate(f_host(1:N,1:M))
  call hipcheck(hipMalloc(f_device, mold=f_host))

  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```

In this example, space is simply allocated for `f_device`. Suppose though that `f_host` has already initialized data on the host side and you would like to allocate and copy data from the host to the device. For this, you can use the `source` argument. This performs the same action as using `mold`, but also perform a `hipMemcpy` from the host to the device after allocating the device data.

```fortran
  use hipfort
  use hipfort_check
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  real, pointer, dimension(:,:) :: f_device
  
  allocate(f_host(1:N,1:M))
  
  ! Initialize f_host
  f_host(1:N,1:M) = 0.0
  
  call hipcheck(hipMalloc(f_device, source=f_host))
  
  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```

If, however, you decide to use a `type(c_ptr)` for your device data, `hipMalloc` expects the pointer and the amount of data, in bytes, that you want to allocate and associate with that pointer. For this, you can use the `SIZEOF` intrinsic in Fortran to obtain the amount of memory required for the host pointer.

```fortran
  use hipfort
  use hipfort_check
  use iso_c_binding
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  type(c_ptr) :: f_device
  
  allocate(f_host(1:N,1:M))
  
  call hipcheck(hipMalloc(f_device, SIZEOF(f_host))
  
  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```

To summarize, we have learned how to allocate and deallocate memory on the host and device using the `hipMalloc` and `hipFree` functions, respectively. We have shown how you can use either `type(c_ptr)` and Fortran `pointer` types for managing either host or device data. In practice, you will most likely find that host data is managed using Fortran `pointer`, rather than `type(c_ptr)`. Whether you use one or the other for the device data depends on your preference or the needs or design criteria of your application. There is no significant difference in performance from this choice. When using Fortran pointers for the device data, however, you have seen that you can use the optional `mold` or `source` arguments in `hipMalloc`. These optional arguments are a convenience and can save you from having to write boiler-plate code for calculating array sizes, shapes, or number of bytes. We now turn our attention to two different types of host memory.

#### Pinned v. pageable memory
By default, using the `allocate` intrinsic for host data results in pageable memory.**Pageable memory** refers to a concept in computer memory management where portions of a program's or system's memory can be temporarily transferred between the main memory (RAM) and secondary storage (usually a hard disk or SSD). This transfer is typically done in the form of pages, which are fixed-size blocks of memory.

In pageable memory systems, not all pages of a program or system need to reside in the physical RAM at all times. Instead, the operating system can swap pages in and out of RAM as needed. This allows more programs to be loaded into memory than can fit simultaneously, and the operating system can prioritize which pages are actively used and keep them in RAM while swapping less frequently used pages to secondary storage.

This paging mechanism helps optimize the use of available memory, allowing for efficient multitasking and the execution of larger programs that might not fit entirely in RAM. However, the process of swapping pages in and out of RAM can introduce some overhead, as data must be transferred between the faster RAM and the slower secondary storage.

**Pinned memory** is a concept in computer memory management where a specific region of memory is prevented from being swapped or moved by the operating system. Unlike pageable memory, which can be moved between RAM and secondary storage, pinned memory remains fixed in physical memory (RAM). In a GPU programming context, **pinned memory** (or page-locked memory, or non-pageable memory) is host memory that is mapped into the address space of all GPUs, meaning that the pointer can be used on both host and device. 

In HIPFort, you can allocate pinned memory using the `hipHostMalloc` function. Like `hipMalloc`, this function takes in the device pointer (either Fortran `pointer` or `type(c_ptr)`) as well as the `size` (in bytes), the `mold`, or the `source`, in addition to a flag that specifies the properties of the allocated memory. Any host memory that is allocated with `hipHostMalloc` must be deallocated using `hipHostFree`.

```fortran
  use hipfort
  use hipfort_check
  use iso_c_binding
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  type(c_ptr) :: f_device
  
  ! Allocate pinned host memory
  call hipcheck(hipHostMalloc(f_host, dims=[N,M],hipHostMallocDefault)
  
  ! Allocate device memory
  call hipcheck(hipMalloc(f_device, SIZEOF(f_host))
  
  ...
  
  call hipcheck(hipHostFree(f_host))
  call hipcheck(hipFree(f_device))
  
```

In this example, we've set the pinned memory allocation flag to `hipHostMallocDefault`, but there are a few options that you can choose from. Below is a list of available options and what they mean.


| Flag | Purpose |
| :--- | :--- |
|hipHostMallocCoherent| Allocate coherent memory, this means the memory is accessible from the host at the same time it is being accessed by a kernel (fine-grained synchronisation), but access is slower. Synchronisation (both host and device receiving all updates to the memory) is *guaranteed* at all stream and device synchronisation calls as well as event synchronisation calls.|
|hipHostMallocNonCoherent| Allocate non-coherent memory for performance, synchronisation between host and device is only guaranteed after a call to **hipStreamSynchronize** or **hipDeviceSynchronize**.|
|hipHostMallocNumaUser| Use the user-defined NUMA policy when allocating memory. This might be useful for architectures like AMD EPYC, where L3 cache is localised to chiplets.|
|hipHostMallocMapped| Map the allocation into the address space for the current device. |
|hipHostMallocPortable | Memory is considered allocated in all contexts. Memory may be portably accessed by all devices.|
|hipHostMallocWriteCombined| Combine writes for performance. Not performant for reads from the CPU however.|
|hipHostMallocDefault| Combination of Mapped, Portable, and Coherent flags, the default. |

The **hipHostMallocCoherent** and **hipHostMallocNonCoherent** flags are mutually exclusive. 


In GPU programming, using pinned memory can be beneficial in certain scenarios to optimize data transfers between the CPU and GPU. Pinned memory, also known as "pinned host memory" or "pinned host arrays," remains fixed in physical memory and can offer advantages in terms of data transfer bandwidth and latency. Here are some situations where you might consider using pinned memory in GPU programming:

* Frequent Data Transfers Between CPU and GPU: If your application involves frequent data transfers between the CPU and GPU, using pinned memory can reduce the overhead associated with pageable memory, as pinned memory is not subject to paging and is more suitable for high-throughput data transfer.

* Direct Memory Access (DMA) Transfers and Asynchronous Operations: Pinned memory is often recommended when using asynchronous data transfers or Direct Memory Access (DMA) operations. Asynchronous transfers allow the CPU and GPU to perform computations concurrently, and pinned memory ensures that data can be efficiently transferred without the need for explicit staging buffers.

* Streamlined Data Movement for HIP or OpenCL Kernels: When working with GPU programming frameworks like CUDA or OpenCL, pinned memory can help streamline data movement. It allows you to pass pointers directly to GPU kernels without the need for additional data copying, resulting in better performance.

* Optimizing Memory Bandwidth: Pinned memory can provide better memory bandwidth compared to pageable memory in scenarios where optimizing memory throughput is crucial. This is particularly relevant when dealing with large datasets or when maximizing data transfer rates is a priority.

* Reducing Latency in Interactive Applications: In interactive applications or simulations where low latency is critical, pinned memory can help minimize the time required for data transfers between the CPU and GPU. This is important for applications that demand real-time responsiveness.


It's important to note that while pinned memory can offer performance benefits in certain situations, it comes with trade-offs. Often, the total amount of pinned memory available may be limited by the system. Therefore, it's recommended to use pinned memory judiciously based on the specific requirements and characteristics of your GPU-accelerated application. In practice, you can find the amount of available page locked (pinned) memory (on Linux operating systems), using `ulimit -a `, and looking for `max locked memory` or `mlock`


#### Memory movement
When explicitly managing host and device pointers in HIPFort, you will need to copy data between host and device memory. This is done using the `hipMemcpy` function. The `hipMemcpy` function has a number of interfaces that are defined so that you can easily work with `type(c_ptr)` or Fortran `pointer` types for host and device data.

For example, when both the source and destination pointers are `type(c_ptr)`

```fortran

status = hipMemcpy( destination, &  ! Destination pointer of type(c_ptr)
                    src, &          ! Source pointer of type(c_ptr)
                    size, &         ! Amount of data to copy in bytes 
                    enum )          ! Memory copy type
```


The `hipMemcpy` function expects the destination pointer (where data is going to), "source pointer" (where data is coming from), the amount of data (in bytes) to copy, and an enumerator that indicates the direction of the memory copy. The enumerator can be one of

* `hipMemcpyHostToDevice`
* `hipMemcpyDeviceToHost`
* `hipMemcpyHostToHost`
* `hipMemcpyDeviceToDevice`

The output of `hipMemcpy` is an `integer(kind(hipsuccess))` status code that should be passed through some kind of error checking.

Alternatively, if both the source and destination pointers are Fortran `pointer`, you can call `hipMemcpy` as simply as

```fortran
status = hipMemcpy( destination, &  ! Destination pointer are Fortran `pointer`
                    src, &          ! Source pointer Fortran `pointer`
                    enum )          ! Memory copy type

```
In this case, we do not need to specify the amount of data that is being copied. Instead, HIPFort will use the size of the destination pointer to determine the amount of data that needs to be copied.

```fortran
  use hipfort
  use hipfort_check
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  real, pointer, dimension(:,:) :: f_device
  
  allocate(f_host(1:N,1:M))
  call hipcheck(hipMalloc(f_device, mold=f_host))

  ...
  
  call hipcheck(hipMemcpy(f_device,f_host,hipMemcpyHostToDevice))
  
  ...
  
  call hipcheck(hipMemcpy(f_host,f_device,hipMemcpyDeviceToHost))

  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```


It's important to notice that the `hipMemcpy` function expects both the source and device pointers to be `type(c_ptr)` or both need to be Fortran `pointer`. If you are using different types for host and device pointers, you will need to obtain a `c_ptr` from the Fortran `pointer` by calling the `c_loc` intrinsic from the `iso_c_binding` module.

```fortran
  use hipfort
  use hipfort_check
  use iso_c_binding
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  type(c_ptr) :: f_device
  
  allocate(f_host(1:N,1:M))
  call hipcheck(hipMalloc(f_device, sizeof(f_host))

  ...
  
  call hipcheck(hipMemcpy(f_device,c_loc(f_host),hipMemcpyHostToDevice))
  
  ...
  
  call hipcheck(hipMemcpy(c_loc(f_host),f_device,hipMemcpyDeviceToHost))

  ...
  
  deallocate(f_host)
  call hipcheck(hipFree(f_device))
  
```

By default, memory movement with `hipMemcpy` is a *blocking* call. This means that host code will not execute until the data is completely moved between host and device; `hipMemcpy` does not simply return when the instructions to move data are dispatched. In some cases however, you may want to overlap data copy with computations using an *asynchronous* memory copy. 

HIP and HIPFort use **streams** to schedule asynchronous operations. When a stream is not specified, kernel launches and memory operations are carried out on the default stream (stream 0). All instructions on the default stream are blocking calls. However, you can create pointers associated with different execution streams. Instructions issued on different streams are permitted to execute in any order, so long as GPU resources are available to execute them. This can be useful when an application has some amount of task parallelism or has the ability to overlap memory movement between host and device with kernel execution.

To perform an asynchronous memory copy, you can use the `hipMemcpyAsync` function. This function is nearly identical to `hipMemcpy` except that it also takes in a `type(c_ptr)` that references a HIP stream as the last argument. It is important to point out that **your host memory must be pinned memory**. If your host memory is pageable, then the memory copy will be synchronous and blocking.

```fortran
  use hipfort
  use hipfort_check
  use iso_c_binding
  
  implicit none
  
  integer, parameter :: N=1000
  integer, parameter :: M=5000
  real, pointer, dimension(:,:) :: f_host
  real, pointer, dimension(:,:) :: f_device
  type(c_ptr) :: stream => c_null_ptr
  
  ! Allocate pinned host memory
  call hipcheck(hipHostMalloc(f_host, dims=[N,M],hipHostMallocDefault)
  
  ! Allocate device memory
  call hipcheck(hipMalloc(f_device, mold=f_host)
  
  ...
  
  ! Create a stream
  call hipcheck(hipStreamCreate(stream))
  
  ...
  
  ! Asynchronous memory copy
  call hipcheck(hipMemcpyAsync(f_device,f_host,hipMemcpyHostToDevice,stream))
  
  ...
  
  call hipcheck(hipHostFree(f_host))
  call hipcheck(hipFree(f_device))
  
```


## Optimizing Memory Transfers between CPU-GPU and GPU-GPU (AMD CDNA2)

When transferring data from a pinned host allocation to a device, HIP usually runs at the limit of the interconnect’s capacity. However, sometimes the interconnect isn't the limiting factor.

To move data on and off a GPU like the MI200, we primarily use the System Direct Memory Access (SDMA) engine. The SDMA engine handles blocks of memory and sends them to the off-device interconnect, whether it’s GPU-CPU or GPU-GPU. Each Graphics Core Die (GCD) has its own SDMA engine for these transfers.

An important thing to note is that SDMA engines work independently from the GPU’s computing tasks. This means that transferring data doesn’t interfere with the GPU’s computing performance, though it can affect memory bandwidth slightly. These engines are optimized for PCIe-4.0 x16, which can handle up to 32 GB/s of data transfer.

One standout feature of the MI250X platform is its Infinity Fabric™ interconnect, which connects the host and device. This interconnect performs better than standard PCIe-4.0, offering roughly 50% more bandwidth. However, the SDMA engine doesn’t run at this higher speed, so it can’t fully utilize the faster interconnect’s bandwidth.

To overcome this bandwidth limitation, you can bypass the SDMA engine and use a “blit” kernel, a type of copy kernel that utilizes the GPU’s compute units. However, this approach uses up compute resources, which might not always be ideal; for example, if you are overlapping compute and data movement through asynchronous operations. To enable blit kernels, you can set the environment variable `HSA_ENABLE_SDMA=0`, which disables the SDMA engine. On systems where the GPU uses a PCIe interconnect rather than an Infinity Fabric interconnect, blit kernels won't impact bandwidth but will still consume compute resources. This choice between SDMA and blit kernels is also relevant for MPI data transfers and GPU-GPU transfers.

## Managed Memory

Up until this point, we have handled host and device memory using explicit memory management. Essentially, we have allocated distinct pointers for host and device memory. To ensure the correct data is available in host or device memory, we used `hipMemcpy` to copy data between the host and device. In what follows, we will look at "managed memory", which provides the means to use a single pointer that is addressable on the host and device. Although this may sound similar to pinned memory, managed memory offers the ability to prefetch data to the device or provide hints to the runtime about how data will be accessed so that kernel performance is optimal.

Managed memory refers to universally addressable, or unified memory available on the MI200 and MI300 series of GPUs. A key benefit of managed memory is that it can help improve programmer productivity by allowing you to use a single pointer for host and device memory. This allows you to take a more incremental approach towards GPU acceleration wherein you can have host and device kernels that operate on the same pointers without having to manually control data movement between the two memory spaces. Performance related to data movement can be improved by prefetching data and by providing hints about data migration. Additionally, managed memory allows you to work on problems wherein the amount of data exceeds the memory capacity of your target GPU. You can effectively use the combined total amount of memory on host and device.


### Verifying your GPU supports managed memory

When writing HIPFort applications that use managed memory, it is wise to check that the target GPU supports this feature. This can be done with the `hipDeviceGetAttribute` method. You may recall that this method accepts three arguments

1. `type(c_ptr) :: val` - A pointer to a value to return
2. `integer :: attr` - The attribute to query
3. `integer :: deviceId` - Which device to query for information

To determine if managed memory is supported, we can use the built in `hipDeviceAttributeManagedMemory` integer enum to specifically query for this feature. The example below shows how we can use this function in practice. First, the `hipDeviceGetAttribute` function is called to obtain the result in the `managed_memory` `c_ptr`. We then convert the c_ptr to a Fortran `integer, pointer` called `supported` which will have a value of 1 if managed memory is supported and 0 otherwise. Note that it is assumed that the `managed_memory` `c_ptr` has been previously allocated. You can see an implementation of this utility in the `supports_managed_memory` function defined in [hip_utils.f90](../Hipfort_Example/hip_utils.f90)

```fortran
use hipfort
use hipfort_check
use iso_c_binding
integer :: dev_id
type(c_ptr) :: managed_memory
integer, pointer :: supported

...
call hipcheck(hipDeviceGetAttribute(managed_memory, &
    hipDeviceAttributeManagedMemory,dev_id)) ! Check if managed memory is supported on this device

call c_f_pointer(managed_memory, supported) ! convert output to c_ptr
...

```


In the event that managed memory is not supported, you can still used the managed memory API. However, all memory allocations will revert to pinned memory allocations, but the behavior of other managed memory API calls can be unpredictable and the performance of GPU kernels may not be acceptable.


### Allocating managed memory
Managed memory can be allocated using the `hipMallocManaged` function, which accepts a Fortran pointer, the dimensions of the pointer, and a managed memory allocation flag. The managed memory allocation flag must be either `hipMemAttachGlobal` or `hipMemAttachHost`. The default, and recommended flag, is  `hipMemAttachGlobal`, which makes the memory accessable by any stream on any device.

```fortran
real,pointer,dimension(:) :: f
integer, parameter :: N = 1000

call hipcheck(hipMallocManaged(f,N,hipMemAttachGlobal))
```

The example above shows how to allocate a `real` single dimension array `f` that is accessible on both the host and device. Note that when passing the size of the array to the `hipMallocManaged` function, the lower bound of the array is set to `1`. 

### Coherence
When working with pointers that are accessible on both the host and device, some subtleties arise due to the fact that GPU kernels are executed asynchronously with respect to the host. For example, consider the following code snippet :

```fortran
use hipfort
use hipfort_check

implicit none

real,pointer,dimension(:) :: f, x
integer, parameter :: N = 1000

call hipcheck(hipMallocManaged(f,N,hipMemAttachGlobal))
call hipcheck(hipMallocManaged(x,N,hipMemAttachGlobal))

call device_subroutine(f, x)

call host_subroutine(f, x)
```

In this example, we have two arrays that are allocated using managed memory. We then call a subroutine that executes a GPU kernel (`device_subroutine`) and then we call a subroutine that executes instructions on the host (CPU). In this case, the GPU kernel is launched from the host and the host continues to execute instructions. Depending how long it takes to complete the execution of the GPU kernel, the `host_subroutine` may be operating on `f` and `x` at the same time as the `device_subroutine`. By default, managed memory allows for such a scenario to occur and this behavior is related to the *coherence* of the managed memory.

In this context, "coherent" means that if a data object is accessed by multiple agents (like the CPU and GPU), each will see exactly the same state. In HIP managed memory we can set the *granularity* of the memory coherence to be either **fine-grained** or **coarse-grained**. 

* Fine-grained coherence (the default behavior) means the coherence is supported while a CPU/GPU kernel is running and implies that up-to-date data may be made visible to others regardless of kernel boundaries as discussed above.
* Coarse-grained coherence means that memory is only considered up to date at kernel boundaries, which can be enforced through `hipDeviceSynchronize`, `hipStreamSynchronize`, or any blocking operation that acts on the default (null) stream.

Using fine-grained coherence is preferable for ensuring correctness in your application while allowing you to think less about the "freshness" of your data on either the host or device. However, acessing or modifying managed memory simultaneously on the host and device can come with some performance penalties for your kernels. Because of this, if you want to enforce coarse-grained coherence, it is advisable to use `hipDeviceSynchronize` (if you are using a single default stream) or `hipStreamSynchronize` (if you are using multiple streams) to block the host from proceeding until your GPU kernels finish runing. In the above example, we could insert a call to `hipDeviceSynchronize` as follows :

```fortran
use hipfort
use hipfort_check

implicit none

real,pointer,dimension(:) :: f, x
integer, parameter :: N = 1000

call hipcheck(hipMallocManaged(f,N,hipMemAttachGlobal))
call hipcheck(hipMallocManaged(x,N,hipMemAttachGlobal))

call device_subroutine(f, x)

call hipcheck(hipDeviceSynchronize())

call host_subroutine(f, x)
```

As a further optimization, we can use the `hipMemAdvise` function to indicate that our managed memory will be used with coarse-grained coherence. Indicating to the runtime that data in managed memory is coarse-grained (coherent at kernel boundaries) allows for further optimizations during kernel execution. The `hipMemAdvise` function expects the following arguments

* [in]	`dev_ptr`	pointer to memory to set the advice for ( `type(c_ptr)` )
* [in]	`count` 	size in bytes of the memory range, it should be CPU page size alligned.
* [in]	`advice`	advice to be applied for the specified memory range
* [in]	`device`	device to apply the advice for

In our example, above, we can insert calls to `hipMemAdvise` to indicate coarse-grained coherence for both `f` and `x` as follows

```fortran
use hipfort
use hipfort_check
use iso_c_binding

implicit none

real,pointer,dimension(:) :: f, x
integer, parameter :: N = 1000

call hipcheck(hipMallocManaged(f,N,hipMemAttachGlobal))
call hipcheck(hipMallocManaged(x,N,hipMemAttachGlobal))

! Set coarse-grained coherence for f on device id 0
call hipcheck(hipMemAdvise(c_loc(f),sizeof(f),hipMemAdviseSetCoarseGrain,0)

! Set coarse-grained coherence for x on device id 0
call hipcheck(hipMemAdvise(c_loc(x),sizeof(x),hipMemAdviseSetCoarseGrain,0)

call device_subroutine(f, x)

call hipcheck(hipDeviceSynchronize())

call host_subroutine(f, x)
```

### Providing other advice about managed memory
At any point in your code, after allocation of managed memory, you can provide advice to the memory management runtime about the coherence, locality, and intended usage of managed memory. Up to this point, we have only looked at how to use `hipMemAdvise` to set the coherence from fine-grained (the default) to coarse-grained using the `hipMemAdviseSetCoarseGrain` advice flag. The table below provides all of the flags that can be used to provide advice about managed memory


| Enumerator | |
| --- | --- |
| hipMemAdviseSetReadMostly | Data will mostly be read and only occassionally be written to |
| hipMemAdviseUnsetReadMostly | Undo the effect of hipMemAdviseSetReadMostly. |
| hipMemAdviseSetPreferredLocation | Set the preferred location for the data as the specified device |
| hipMemAdviseUnsetPreferredLocation | Clear the preferred location for the data. |
| hipMemAdviseSetAccessedBy | Data will be accessed by the specified device so prevent page faults as much as possible |
| hipMemAdviseUnsetAccessedBy | Let HIP to decide on the page faulting policy for the specified device |
| hipMemAdviseSetCoarseGrain | The default memory model is fine-grain. That allows coherent operations between host and device, while executing kernels. The coarse-grain can be used for data that only needs to be coherent at dispatch boundaries for better performance |
| hipMemAdviseUnsetCoarseGrain | Restores cache coherency policy back to fine-grain. At any point, you can change the coherence back to fine-grained using the `hipMemAdviseUnsetCoarseGrain` advice flag |

### Prefetching
In addition to setting memory coherence and other advice about managed memory usage, data can be prefeteched to the target device, including the CPU (host), at any point after managed memory has been allocated. Prefetching helps reduce page faults, which can significantly improve performance. For best possible performance, it is advisable to prefetch data to the appropriate device when you know it is needed for read operations.

You can prefetch managed memory using the `hipMemPrefetchAsync` call, which expects the following arguments
* [in]	dev_ptr	pointer to be prefetched (`type(c_ptr)`)
* [in]	count	size in bytes for prefetching
* [in]	device	destination device to prefetch to
* [in]	stream	stream to enqueue prefetch operation (use `0` for the default stream)


Continuing with our example from above, suppose we know that the `device_subroutine` uses `x` to compute `f`; in other words values of `x` are read from in order to compute and write values of `f`. In this case, we could prefetch `x` prior to the `device_subroutine` call to the device we are working on.


```fortran
use hipfort
use hipfort_check
use iso_c_binding

implicit none

real,pointer,dimension(:) :: f, x
integer, parameter :: N = 1000

call hipcheck(hipMallocManaged(f,N,hipMemAttachGlobal))
call hipcheck(hipMallocManaged(x,N,hipMemAttachGlobal))

! Set coarse-grained coherence for f on device id 0
call hipcheck(hipMemAdvise(c_loc(f),sizeof(f),hipMemAdviseSetCoarseGrain,0)

! Set coarse-grained coherence for x on device id 0
call hipcheck(hipMemAdvise(c_loc(x),sizeof(x),hipMemAdviseSetCoarseGrain,0)

...

! Pre-fetch `x` to device id 0 on stream 0
call hipcheck(hipMemPrefetchAsync(c_loc(x),sizeof(x),0,0))

call device_subroutine(f, x)

call hipcheck(hipDeviceSynchronize())

call host_subroutine(f, x)
```

In this example, since we are using the default stream, the kernel launch in `device_subroutine` will not occur until the `hipMemPrefetchAsync` call is complete; if we wanted to make it asynchronous, we would use a different stream. When the `device_subroutine` is executed, all of the necessary page migration has been completed for `x` and we should expect no page faults and the best possible performance, approaching that achieved with explicit memory movement.